In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
df_age_gender=pd.read_csv('/Users/dinamaizlis/Downloads/age_gender.csv')

## data

In [ ]:
df_age_gender.shape

In [ ]:
df_age_gender.isnull().sum()

In [ ]:
df_to_show=df_age_gender.copy()
df_to_show['pixels']=df_age_gender['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

In [ ]:
df_to_show.head(3)
s=df_to_show['pixels']
print(df_to_show['pixels'][0][1])
dfn = df_to_show['pixels'].convert_dtypes()
type(dfn[0][1])

In [ ]:
X = np.array(df_to_show['pixels'].tolist())
X = X.reshape(X.shape[0],48,48,1)

X.shape

In [ ]:
plt.figure(figsize=(10,10))
for i , image in enumerate(np.random.randint(0,23705,16)):
    plt.subplot(4,4,i+1)
    plt.imshow(X[image], cmap = "gray")
    plt.axis("off")
plt.show()

In [ ]:
sns.barplot(y=list(df_to_show['gender'].value_counts().values),x= list(df_to_show['gender'].value_counts().index))

### split the the "pixels" feature ->for each pixel feature


In [ ]:
df_pixels=df_age_gender.pixels.str.split(expand=True,)

In [ ]:
d=df_pixels.astype(str).astype(int)

In [ ]:
df_pixels=d/255

In [ ]:
df_pixels_age=df_pixels.copy()
df_pixels_age=df_pixels_age.join(df_to_show)

In [ ]:
df_pixels_gender=df_pixels_age.drop(['pixels','age','img_name','ethnicity'],1)

In [ ]:
df_pixels_gender.shape

In [ ]:
df_pixels_gender.head(4)

## train&test
80% train  and 20% test



In [ ]:
mask = np.random.rand(len(df_pixels_gender)) <= 0.8
training_data = df_pixels_gender[mask]
testing_data = df_pixels_gender[~mask]

In [ ]:
X_train=training_data.drop(["gender"],1)
y_train=training_data["gender"]

In [ ]:
X_test=testing_data.drop(["gender"],1)
y_test=testing_data["gender"]

#### import tensorflow1

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## Logistic Regression in Tensorflow


In [ ]:
W = tf.Variable(tf.random_normal(shape=[2304, 1]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

X= tf.placeholder(dtype=tf.float32, shape=[None, 2304])
y = tf.placeholder(dtype=tf.float32, shape=[None, 1])

In [ ]:
learning_rate = 0.001
batch_size =800
iter_num = 1500

In [ ]:
loss_trace = []
train_acc = []
test_acc = []

mod = tf.matmul(X, W) + b
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=mod, labels=y))
opt = tf.train.GradientDescentOptimizer(learning_rate)
update = opt.minimize(loss)
prediction = tf.round(tf.sigmoid(mod))
correct = tf.cast(tf.equal(prediction, y), dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y), dtype=tf.float32))

for epoch in range(iter_num):
    batch_train_X=[]
    batch_train_y=[]
    import random
    for i in range(batch_size):
        n = random.randint(0,X_train.shape[0]-1)
        batch_train_X.append(X_train.iloc[n])
        batch_train_y.append(y_train.iloc[n])
    batch_train_X=np.matrix(batch_train_X)
    batch_train_y = np.matrix(batch_train_y).T
    sess.run(update, feed_dict={X: batch_train_X, y: batch_train_y})
    
    temp_loss = sess.run(loss, feed_dict={X: batch_train_X, y: batch_train_y})
    temp_train_acc = sess.run(accuracy, feed_dict={X: X_train, y: np.matrix(y_train).T})
    temp_test_acc = sess.run(accuracy, feed_dict={X: X_test, y: np.matrix(y_test).T})
    
    loss_trace.append(temp_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
    if (epoch + 1) % 10 == 0:
        print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss,
                                                                          temp_train_acc, temp_test_acc))
     

## Neural Network

In [ ]:
def Neural_Network(hidden):   
    hidden_layer_nodes = hidden
    x = tf.placeholder(tf.float32, [None, 2304])
    y_ = tf.placeholder(tf.float32, [None, 1])
    W1 = tf.Variable(tf.truncated_normal([2304,hidden_layer_nodes], stddev=0.1))
    b1 = tf.Variable(tf.constant(0.1, shape=[hidden_layer_nodes]))
    z1 = tf.add(tf.matmul(x,W1),b1)
    a1 = tf.nn.relu(z1)
    W2 = tf.Variable(tf.truncated_normal([hidden_layer_nodes,1], stddev=0.1))
    b2 = tf.Variable(0.)
    z2 = tf.matmul(a1,W2) + b2
    pred = 1 / (1.0 + tf.exp(-z2))
    pred2 = 1-(1 / (1.0 + tf.exp(-z2)))
    loss = tf.reduce_mean(-(y_*tf.log(pred)+(1-y_)*tf.log(pred2)))
    update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
    data_x = X_train
    data_y = np.matrix(y_train).T
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    for i in range(0,5):
            sess.run(update, feed_dict = {x:data_x, y_:data_y})
    predict=pred.eval(session=sess, feed_dict = 	{x:X_test})
    #print(predict)
    count=0
    y_test_new=np.matrix(y_test).T
    #accuracy
    for i in range (predict.shape[0]):
        if predict[i]>0.5:
            predict[i]=0
        else:
            predict[i]=1
  
        if(predict[i]-y_test_new[i]==[0.]):
            count=count+1
    return count


In [ ]:
max_hidden=0
max_val=0
for i in range (100):
    for j in range (5):
        if max_val< Neural_Network(i+4):
            max_val= Neural_Network(i+4)
            max_hidden=i+4
    if i%10==0:
        print ("max_val  ", max_val,"max_hidden  ",max_hidden)